In [2]:
import torch
import torch.utils.data as data
import os
import pickle
import numpy as np
import nltk
from PIL import Image
import os.path
import random
from torch.autograd import Variable
import torch.nn as nn
import math
import cv2
from torchvision import datasets, transforms
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [18]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64 , 128 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 , 256 , 8)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 8),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

def data_loader(N = 200, NP = 200):
    
    
    autoencoder = Autoencoder()
    Q = autoencoder.encoder
    Q.load_state_dict(torch.load(r"C:\Users\Navdeep\Downloads\cae_encoder.pkl"))
    if torch.cuda.is_available():
        Q = Q.to(torch.device("cuda"))

    encoded_w_m=np.zeros((N,256),dtype=np.float32)
    for image_number in range(0,N):
#         print(image_number)
        image= cv2.imread(f'C:/Users/Navdeep/Downloads/images/images/{image_number}.jpg' , cv2.IMREAD_GRAYSCALE)
        to_tensor = transforms.ToTensor()
        torch_img = to_tensor(image)
        if torch.cuda.is_available():
            torch_img = torch_img.to(torch.device("cuda"))  # Move input tensor to GPU

        output=Q(torch_img)
        output = output.squeeze()
        output=output.data.cpu()
        output = output.numpy()
        encoded_w_m[image_number] = output
    
    
    
    example = pd.read_csv(r"C:\Users\Navdeep\Downloads\output_modified3.csv")
    #print(len(example))
    index1 = 0
    index2 = 0
    index3 = 1
    path_lengths=np.zeros((N,NP + 1),dtype=np.float32)
    prev_image_number = 189.0
    max_length=0
    while index1 < len(example) :
        row = example.iloc[index1]
        index_start = row['index_start']
        index_goal = row['index_goal']
        len_path = (index_goal - index_start) + 1
        image_number = row['image_number']
        index1 = index_goal + 1
        index1 = int(index1)
        if (image_number == prev_image_number):
            path_lengths[index2][0] = image_number
            path_lengths[index2][index3] = len_path
            index3 = index3 + 1
        else :
            index3 = 1
            index2 = index2 + 1
            path_lengths[index2][0] = image_number
            path_lengths[index2][index3] = len_path
            prev_image_number = image_number
            index3 = index3 + 1    
        if len_path > max_length :
            max_length = len_path
            
    
    max_length = int(max_length)
    
    
    path_lengths = path_lengths[:, 1:]
    
    
    
    paths=np.zeros((N,NP,max_length,2), dtype=np.float32)
    index = 0
    index1 = 0
    index2 = 0
    index3 = 0
    prev_image_number = 189.0
    while(index < len(example)) :
        row = example.iloc[index]
        index_start = int(row['index_start'])
        index_goal = int(row['index_goal'])
        image_number = row['image_number']
        index = index_goal + 1
        if (image_number == prev_image_number):
            index3 = 0
            for i in range(index_start , index_goal + 1):
                row = example.iloc[i]
                x = row['current_x']
                y = row['current_y']
                #print([x , y])
                paths[index1][index2][index3] = [x , y]
                #paths[index1][index2][index3][1] = y
                index3 = index3 + 1
            index2 = index2 + 1  
        else :
            index1 = index1 + 1
            index2 = 0
            index3 = 0
            for i in range(index_start , index_goal + 1):
                row = example.iloc[i]
                x = row['current_x']
                y = row['current_y']
                #print([x , y])
                paths[index1][index2][index3] = [x , y]
                #paths[index1][index2][index3][1] = y
                index3 = index3 + 1
            prev_image_number = image_number    
            index2 = index2 + 1
    
    
    
    
    dataset=[]
    targets=[]
    for i in range(200):
        for j in range(200):
            if path_lengths[i][j] > 0 :
                for m in range(0, int(path_lengths[i][j]-1)):
                    data=np.zeros(260,dtype=np.float32)
                    for k in range(0 , 256):
                        data[k] = encoded_w_m[i][k]
                    data[256] = paths[i][j][m][0]
                    data[257] = paths[i][j][m][1]
                    data[258] = paths[i][j][int(path_lengths[i][j]) - 1][0]
                    data[259] = paths[i][j][int(path_lengths[i][j]) - 1][1]
                    
                    targets.append(paths[i][j][m+1])
                    dataset.append(data)
                    
    #data = list(zip(dataset, targets))
    #random.shuffle(data)
#    dataset,targets=zip(*data)
    
    return np.asarray(dataset),np.asarray(targets)
    
    
    
dataset , targets = data_loader()


# print(max_length)

In [20]:
print(dataset[0])

[ 4.04848717e-03  3.13273910e-03  3.86151765e-03 -3.32042063e-03
  2.55635940e-02 -7.39802327e-03 -2.41143107e-02  1.56192109e-02
  9.43895429e-05 -2.87109986e-03 -7.06559839e-03 -1.39839482e-03
 -6.63321558e-03 -2.10166257e-02 -2.95215193e-03 -1.36117395e-02
  5.12059452e-03  7.30101578e-03 -8.39279499e-04  3.07599921e-03
  4.43638489e-03 -1.30338538e-02  7.81337451e-03  1.01922182e-02
  6.30492344e-03  4.25265357e-03  9.78552271e-05 -9.77579504e-03
  4.37315926e-03 -1.59082934e-03  1.51448073e-02 -7.09948875e-03
  3.18835978e-03  7.13348063e-03 -1.94739476e-02 -1.42549518e-02
 -7.37990811e-03  1.33055262e-02  1.41462535e-02 -5.77224046e-03
  5.37839811e-03 -7.29449838e-03 -9.23321303e-03 -2.84373853e-03
  7.39513384e-03 -1.89395230e-02 -5.95453661e-04  6.11360138e-03
 -2.08768677e-02  1.45288063e-02  1.20089408e-02  1.93535211e-03
 -9.56982840e-03 -8.60150252e-03  7.84465019e-03 -3.67317535e-03
  6.47991151e-03 -3.75598064e-03  1.29150767e-02 -1.16936979e-04
  1.20504992e-02  6.18846

In [22]:
print(targets[0])

[51.86109  55.634155]


In [21]:
import pandas as pd
import numpy as np
example = pd.read_csv(r"C:\Users\Navdeep\Downloads\output_modified3.csv")


In [25]:
dataset=[]
targets=[]



In [32]:
paths_list = []
for i in range (200) :
    for j in range(200):
        paths_list.append(paths[i][j])


# matrix = [
#     [1, 0, 3],
#     [0, 0, 0],
#     [4, 5, 0],
#     [0, 0, 0],
#     [6, 7, 8]
# ]

# Count non-zero rows using list comprehension
maximum = 0
for matrix in paths_list:
    non_zero_rows_count = sum(1 for row in matrix if any(row))
    if non_zero_rows_count > maximum :
        maximum = non_zero_rows_count

print("Number of non-zero rows:", maximum)

Number of non-zero rows: 57


In [31]:
print(paths_list[0])

[[55.       64.      ]
 [51.86109  55.634155]
 [55.618298 52.335155]
 [55.15355  47.3568  ]
 [59.29151  44.55015 ]
 [62.62616  40.82455 ]
 [67.17638  38.75198 ]
 [70.64894  35.15458 ]
 [69.665344 30.25228 ]
 [74.42799  28.73007 ]
 [79.153145 27.095152]
 [76.337685 22.963182]
 [78.029854 18.258234]
 [74.05808  15.220966]
 [69.20447  14.019955]
 [64.69739  16.184732]
 [62.014683 20.404102]
 [57.063114 19.709871]
 [58.       30.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.        0.      ]
 [ 0.      

In [ ]:
msg.position.x = 0.4082701802253723;
msg.position.y = -0.5790671110153198;
msg.position.z = 0.244498148560524;

In [ ]:
msg.orientation.w = -2.5082286811084487e-05;
msg.orientation.x = 1;
msg.orientation.y = 0.00016352070088032633;
msg.orientation.z = 1.1314374205539934e-05;